In [1]:
import os
os.getcwd()

'D:\\Python\\finominaScraping'

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

In [3]:
def transformSTG_X(path:str,QUARTER_REF:str=5):
    
    DF_RWST = pd.read_csv(path)
    DF_RWST = DF_RWST.rename(columns={
        'Unnamed: 0'            : "Date", 
        'รายได้รวม'              : "TotalRevenue", #*
        'การเติบโตต่อไตรมาส (%)'  : "growth/quarter",
        'การเติบโตต่อปี (%)'       : "growth/year",
        'กำไรสุทธิ'               : "NetProfit",   #*
        'กำไรต่อหุ้น (EPS)'        : "EPS",
        'ROA (%)'               : "ROA",
        'ROE (%)'               : "ROE",
        'อัตรากำไรขั้นต้น (%)'      : "GrossProfitMargin",
        'อัตราส่วนการขายและบริหารต่อรายได้ (%)'  : "OperatingMargin",
        'อัตรากำไรสุทธิ (%)'        : "NetProfitMargin",
        'หนี้สิน/ทุน (เท่า)'         : "debt/capital",
        'วงจรเงินสด (วัน)'          : "CashConversionCycle",
        'มูลค่าหลักทรัพย์ตามราคาตลาด (ล้านบาท)'  : "MKT.values",
        'P/E (เท่า)'                : "P/E",
        'P/BV (เท่า)'               : "P/BV",
        'มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท)' : "BookValue/Share",
        'อัตราส่วนเงินปันผลตอบแทน'    : "DividendYield", #drop
        'EV / EBITDA'              : "EV/EBITDA",
        'ราคาล่าสุด (บาท)'           : "Price",     #*
    })
    
    #adj Price with Dv
    DF_RWST['Price'] = DF_RWST['DividendYield'] + DF_RWST['Price']
    #drop Dv
    DF_RWST = DF_RWST.drop('DividendYield',axis=1)
    
    DF_RWST['year'] = DF_RWST['Date'].str.slice(start=2)
    DF_RWST['quarter'] = DF_RWST['Date'].str.slice(stop=1)
    
    #take diff
    DF_RWST = DF_RWST.sort_values(['year','quarter'],ascending=True)
    DF_RWST["TotalRevenue"] = DF_RWST["TotalRevenue"].diff()
    DF_RWST["NetProfit"] = DF_RWST["NetProfit"].diff()
    DF_RWST["Price"] = DF_RWST["Price"].diff()
    
    #transform
    DF_RWST = DF_RWST.sort_values(['year','quarter'],ascending=False)

    DF_Cross = DF_RWST[ ["Symbol","Market","IndusGroup","BusinessGroup"] ]
    
    ##### transformDate #####   
    DF_Date     = pd.DataFrame()
    DF_Date['year']   = DF_RWST['year']
    DF_Date['quarter']= DF_RWST['quarter']
    
    #--tran พศ to คศ  
    year = DF_RWST['year'].to_numpy()
    year = year.astype(int) - 543
    DF_Date['Date'] = np.add(   
        np.add(DF_RWST['quarter'].astype(str),["Q"]),
        year.astype(str) )

    
    
    DF_Ts    = DF_RWST.drop( columns=list(DF_Cross.columns)+list(DF_Date.columns) )
    DF_Ts    = DF_Ts.reset_index(drop=True)
    
    ##### transformDF #####
    STG_symbol = pd.DataFrame()
    for i,c1 in DF_Ts.iterrows():
        transformDF = pd.DataFrame([c1.values], columns=c1.index).add_suffix(f'[0]') 
        ref = 0
        for j,c2 in DF_Ts[i+1:i+1+QUARTER_REF].iterrows():
            ref += 1
            transformDF = pd.concat([transformDF,pd.DataFrame([c2.values], columns=c2.index).add_suffix(f'[-{ref}]')  ] , axis=1)

        STG_symbol = pd.concat([STG_symbol,transformDF], axis=0) 
    STG_symbol = STG_symbol.reset_index(drop=True)
    
    # ADD "year","quarter"
    # STG_symbol = pd.concat([STG_symbol,DF_Date[ ["year","quarter"] ]   ], axis=1) 

    # ADD CrossSection data
    STG_X = pd.concat([STG_symbol,DF_Cross   ], axis=1) 
    
    DF_Date = DF_Date.reset_index()
    STG_X['Date'] = DF_Date['Date']
    
    return STG_X.set_index('Date') ,DF_Date.set_index('Date')
    

In [4]:
def transformSTG_Y(STG_X,Date):
    Y={
        'Price[1]':[np.nan]*1, #1q
        'Price[2]':[np.nan]*2, #2q
        'Price[3]':[np.nan]*3, #3q
        'Price[4]':[np.nan]*4  #4q
    }

    price = STG_X['Price[0]'].values

    for i in range(len(STG_X)):
        for j in range( 1,len(Y)+1 ):
            if i >= j:
                Y[f'Price[{j}]'] += [price[i-j]]    
                
    Y = pd.DataFrame(Y)
    Y['Price[0]'] = STG_X['Price[0]'].values
    Y = Y[Y.columns.sort_values()]
    Y['Date'] = Date.index
    return Y.set_index("Date")

In [12]:
def ratingScore(STG_Y):
    STG_Y['Score'] = 0
    
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]), 'Score'] = STG_Y['Score']+1
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]) & (STG_Y["Price[2]"] > STG_Y["Price[1]"]), 'Score'] = STG_Y['Score']+1
    STG_Y.loc[ (STG_Y["Price[1]"] > STG_Y["Price[0]"]) 
              & (STG_Y["Price[2]"] > STG_Y["Price[0]"])
              & (STG_Y["Price[2]"] > STG_Y["Price[1]"]), 'Score'] = STG_Y['Score']+1
    
    #add
    STG_Y.loc[  (STG_Y["Price[1]"]  >=10) , 'Score'] = STG_Y['Score']+1
    STG_Y.loc[  (STG_Y["Price[2]"]  >=10) , 'Score'] = STG_Y['Score']+1
    return STG_Y

In [13]:
# RWST => transformSTG_Y => STG_X,DATE
STG_X,Date = transformSTG_X('./data/RWST/MFC.csv',)

In [14]:
# STG_X => transformSTG_Y => STG_Y
STG_Y = transformSTG_Y(STG_X,Date)

In [15]:
# STG_Y => Score => score_Y
score_Y = ratingScore(STG_Y)

In [16]:
score_Y.head()

,Price[0],Price[1],Price[2],Price[3],Price[4],Score
Date,,,,,,
3Q2022,-0.09,NaN,NaN,NaN,NaN,0
2Q2022,-4.51,-0.09,NaN,NaN,NaN,1
1Q2022,5.79,-4.51,-0.09,NaN,NaN,0
4Q2021,3.52,5.79,-4.51,-0.09,NaN,1
3Q2021,2.13,3.52,5.79,-4.51,-0.09,3


In [17]:
score_Y

,Price[0],Price[1],Price[2],Price[3],Price[4],Score
Date,,,,,,
3Q2022,-0.09,NaN,NaN,NaN,NaN,0
2Q2022,-4.51,-0.09,NaN,NaN,NaN,1
1Q2022,5.79,-4.51,-0.09,NaN,NaN,0
4Q2021,3.52,5.79,-4.51,-0.09,NaN,1
3Q2021,2.13,3.52,5.79,-4.51,-0.09,3
2Q2021,1.26,2.13,3.52,5.79,-4.51,3
1Q2021,2.08,1.26,2.13,3.52,5.79,0
4Q2020,0.50,2.08,1.26,2.13,3.52,1
3Q2020,0.09,0.50,2.08,1.26,2.13,3


In [ ]:
STG_X.iloc[[0:10]]

In [ ]:
STG_X['Price[0]'].diff()

In [ ]:
for i in STG_X.columns:
    print(i)

In [ ]:
STG_Y["Price[1]"] < STG_Y["Price[0]"]

In [ ]:
np.where(STG_Y["Price[1]"] >= STG_Y["Price[0]"], 1,
         
         
         
         
         99)

In [ ]:
df.iloc[lambda x: x.index % 2 == 0]

In [ ]:
#get set index
SET = pd.read_csv('./data/SET.BK.csv')


In [ ]:
dateSplit = pd.DataFrame()
dateSplit[['day','quarter','year']]=SET['Timestamp'].str.split("-",n=0,expand=True)

In [ ]:
dateSplit['quarter']=dateSplit['quarter'].replace("01","4")
dateSplit['quarter']=dateSplit['quarter'].replace("04","1")
dateSplit['quarter']=dateSplit['quarter'].replace("07","2")
dateSplit['quarter']=dateSplit['quarter'].replace("10","3")

In [ ]:
dateSplit['year'] = pd.to_datetime(dateSplit['year'].str.strip().astype(str), format='%y').dt.strftime('%Y')

In [ ]:
dateSplit['Date'] = np.add(   
    np.add(dateSplit['quarter'].astype(str),["Q"]),
    dateSplit['year'].astype(str) )

In [ ]:
dateSplit['Date'].sort_values('',ascending=False)

In [ ]:
dateSplit['year'].str.strip().strftime('%Y')

In [ ]:
Y = pd.DataFrame( {"Date":q[1].str.strip()} )

In [ ]:
Y

In [ ]:
ratingScore(STG_Y,Date)

In [ ]:
len(STG_X)

In [ ]:
STG_Y.head(3)

In [ ]:
STG_X.head(3)